```{contents}
```
## File System Tools


File system tools are **programmatic interfaces** that allow applications or agents to **read, write, list, search, and manage files and directories** on local or remote storage.
In LLM/agent systems, they provide **grounded access to documents, logs, configs, and artifacts**.

```
LLM / App / Agent  →  File System Tool  →  Files / Folders  →  Content / Metadata
```

---

### Why File System Tools Are Important

* Access **documents as source-of-truth**
* Enable **local RAG** without a database
* Support **stateful workflows** (logs, checkpoints)
* Allow **automation** (ETL, ingestion, backups)
* Reduce hallucinations by grounding on files

---

### Common File System Tool Capabilities

#### Read Operations

* Read text / JSON / CSV / PDF
* Load configs, prompts, policies

#### Write Operations

* Save outputs
* Persist agent memory snapshots
* Generate reports

#### Navigation & Metadata

* List directories
* Search files
* File size, timestamps, permissions

---

### File System Tool Anatomy

A file system tool usually defines:

* **Root path / sandbox**
* **Allowed operations** (read-only vs read-write)
* **Input schema** (path, pattern)
* **Output schema** (content, metadata)
* **Security constraints**

Example:

```
Input : { "path": "docs/incident.md" }
Output: { "content": "...", "size": 2048 }
```

---

### Basic File System Tool (Python)

#### Read File Tool

```python
from pathlib import Path

def read_file(path: str) -> str:
    return Path(path).read_text(encoding="utf-8")
```

#### Write File Tool

```python
def write_file(path: str, content: str):
    Path(path).write_text(content, encoding="utf-8")
```

These functions are the **core primitives** of file system tools.

---

### File System Tool as an API (FastAPI)

Using FastAPI to expose file access safely:

```python
from fastapi import FastAPI, HTTPException
from pathlib import Path

app = FastAPI()
BASE_DIR = Path("./sandbox")

@app.get("/fs/read")
def fs_read(file_path: str):
    full_path = BASE_DIR / file_path
    if not full_path.exists():
        raise HTTPException(status_code=404, detail="File not found")
    return {"content": full_path.read_text()}

@app.post("/fs/write")
def fs_write(file_path: str, content: str):
    full_path = BASE_DIR / file_path
    full_path.parent.mkdir(parents=True, exist_ok=True)
    full_path.write_text(content)
    return {"status": "written"}
```

This creates a **controlled file system tool**.

---

### File System Tools in LLM Workflows

![Image](https://community.intersystems.com/sites/default/files/inline/images/images/image%289789%29.png?utm_source=chatgpt.com)

![Image](https://images.ctfassets.net/0uuz8ydxyd9p/26t4SwsjnUJHpJy8vyZOAM/8357530cbb4df285125bcf0e30ecaedb/Screenshot_2025-07-07_at_3.04.27%C3%A2__PM.png?utm_source=chatgpt.com)

![Image](https://weaviate.io/assets/images/fig2-d0febceaefdcc84b555caad8bfd0b1f2.png?utm_source=chatgpt.com)

**Flow**

1. User asks about a document
2. LLM selects file system tool
3. Tool reads file content
4. Content is injected into context
5. LLM produces grounded response

---

### File System Tool in LangChain

Using LangChain:

```python
from langchain.tools import Tool
from pathlib import Path

def read_local_doc(path: str):
    return Path(path).read_text()

fs_tool = Tool(
    name="LocalFileReader",
    func=read_local_doc,
    description="Read local documents from file system"
)
```

The agent can now **decide when to read a file**.

---

### File System Tools for RAG (Local Documents)

#### Load Documents

```python
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader("./docs", glob="**/*.md")
documents = loader.load()
```

#### Use in Retrieval

* Chunk files
* Embed content
* Retrieve relevant sections

This is **file-based RAG**, often used before vector DBs.

---

### File System Tools vs Database Tools

| Aspect     | File System Tool | Database Tool   |
| ---------- | ---------------- | --------------- |
| Structure  | Unstructured     | Structured      |
| Latency    | Very low         | Low             |
| Query      | Path-based       | SQL / Vector    |
| Versioning | Manual           | Built-in        |
| Best for   | Docs, logs       | Records, memory |

---

### Real-World Agentic Use Case

**IT Support Agent**

* File system tool → Read runbooks (PDF/MD)
* File system tool → Load SLA policies
* Vector DB → Similar incidents
* API tool → Escalation

---

### Security & Production Practices

* Sandbox root directory
* Read-only tools for agents
* Path traversal protection
* File size limits
* Audit logs

---

### Key Takeaways

* File system tools connect agents to **local truth**
* They are essential for document-heavy workflows
* Ideal for ingestion, RAG, logs, configs
* Often combined with vector DBs and APIs
